In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator as Calculator
from numpy import linalg as LA
from rdkit.Chem import Draw

ImportError: No module named 'rdkit'

In [ ]:
data = pd.DataFrame.from_csv('cations.csv')

In [ ]:
data.head()

In [ ]:
n = data.shape[0]

In [ ]:
f = open('Deslist','r')
Deslist = []
for line in f:
    Deslist.append(line.strip('\n\t'))
#print(Deslist)
calc = Calculator(Deslist)
d = len(Deslist)
print(n,d)

In [ ]:
X = np.zeros((n,d+1))
for i in range(n):
    cation = Chem.MolFromSmiles(data['SMILE'][i])
    X[i][:d]    = calc.CalcDescriptors(cation)
    X[i][d] = i
print(X.shape)

In [ ]:
n_train = int(n*0.8)
n_test  = n - n_train
X = np.random.permutation(X)

X_train = np.zeros((n_train,d))
X_test  = np.zeros((n_test,d))
Y_train = np.zeros((n_train))
Y_test = np.zeros((n_test))

X_train[:] = X[:n_train,:-1]
Y_train[:] = X[:n_train,-1]

X_test[:] = X[n_train:,:-1]
Y_test[:] = X[n_train:,-1]

print(X_train.shape)
print(X_test.shape)

In [ ]:
k = 4
itr = 0
cent =np.copy(X_train[:k,:])
C = np.zeros((n_train))  
flag = True
ITR = 15
squareloss = np.zeros((ITR+1))
while flag:
    cn = np.zeros(k)
    csum = np.zeros((k,d))
    flag = False
    for i in range(n_train):
        minm = 1e15
        for j in range(k):
            if (LA.norm(X_train[i]-cent[j])<minm):
                minm = LA.norm(X_train[i]-cent[j])
                m = j
        if C[i] != m :
            flag = True
        C[i] = m
        cn[m] += 1
        csum[m] += X_train[i]
        squareloss[itr] += np.square(LA.norm(X_train[i]-cent[m]))
    if itr == ITR:
        break
    for i in range(k):
        cent[i] = csum[i] / cn[i] 
    itr += 1   

In [ ]:
print(squareloss)
plt.plot(np.arange(0,ITR+1),np.log(squareloss), color="blue", label="Squared resonstruction error")
plt.xlabel('iteration')
plt.ylabel('error')
plt.legend(loc= "upper right")
plt.show()

In [ ]:
for i in range(k):
    cn[i] = 0
    cation = []
    anion  = []
    for j in range(n_train):
        if C[j] == i :
            cn[i] += 1
            cation.append(data['SMILE'][Y_train[j]])
    cation = sorted(cation)
    anion = sorted(anion)
    for j in range(int(cn[i])):
        if j == 0 :
            m = Chem.MolFromSmiles(cation[j])
            Draw.MolToFile(m,'./pic/{}cation{}.png'.format(i,j))
        if j>0:
            if cation[j]!=cation[j-1]:
                m = Chem.MolFromSmiles(cation[j])
                Draw.MolToFile(m,'./pic/{}cation{}.png'.format(i,j))

In [ ]:
for i in range(k):
    print("Cluster",i)
    cn[i] = 0
    clist = []
    for j in range(n_train):
        if C[j] == i :
            cn[i] += 1
            clist.append(data['label'][Y_train[j]]+'   '+data['NAME'][Y_train[j]])  
    csort = sorted(clist)
    for j in range(int(cn[i])):
        print(csort[j])